In [18]:
pip install mysql-connector-python qrcode[pil]


Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install fpdf


Note: you may need to restart the kernel to use updated packages.


In [20]:
import mysql.connector
import qrcode
import os
from fpdf import FPDF

# Conexión a la base de datos
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='ZZW75$Ry32N12',
    database='inedb'
)

cursor = connection.cursor(dictionary=True)

# Crear la carpeta CodigosQR si no existe
os.makedirs('CodigosQR', exist_ok=True)

# Clase para crear un PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Documentos de Identificación', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def link(self, x, y, w, h, link):
        self.set_xy(x, y)
        self.cell(w, h, '', 0, 1, 'L', link)

def generar_qr_y_pdf():
    try:
        # Consulta de datos de la tabla usuarios
        cursor.execute("SELECT * FROM usuarios")
        rows = cursor.fetchall()

        for row in rows:
            # Generar el código QR
            qr = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_L,
                box_size=10,
                border=4,
            )
            qr.add_data(row['curp'])
            qr.make(fit=True)

            # Guardar el código QR en la carpeta CodigosQR
            img = qr.make_image(fill='black', back_color='white')
            qr_filename = f"CodigosQR/qr_{row['id']}.png"
            img.save(qr_filename)

            # Crear el PDF para CURP
            pdf = PDF()
            pdf.add_page()
            pdf.set_font('Arial', 'B', 16)
            pdf.cell(0, 10, f"CURP: {row['curp']}", ln=True)
            pdf.cell(0, 10, f"Nombre: {row['nombre']} {row['apellido_paterno']} {row['apellido_materno']}", ln=True)
            
            # Enlaces a documentos
            pdf.set_font('Arial', 'I', 12)
            pdf.cell(0, 10, 'Consultar CURP en: https://www.gob.mx/curp/', ln=True)
            pdf.link(10, pdf.get_y() - 10, 0, 10, 'https://www.gob.mx/curp/')
            pdf.cell(0, 10, 'Consultar RFC en: https://www.sat.gob.mx/home', ln=True)
            pdf.link(10, pdf.get_y() - 10, 0, 10, 'https://www.sat.gob.mx/home')
            pdf.cell(0, 10, 'Consultar Comprobante de Domicilio en: https://app.cfe.mx/Aplicaciones/CCFE/SolicitudesCFE/Solicitudes/ConsultaTuReciboLuzGmx', ln=True)
            pdf.link(10, pdf.get_y() - 10, 0, 10, 'https://app.cfe.mx/Aplicaciones/CCFE/SolicitudesCFE/Solicitudes/ConsultaTuReciboLuzGmx')
            pdf.cell(0, 10, 'Consultar Acta de Nacimiento en: https://www.gob.mx/ActaNacimiento/', ln=True)
            pdf.link(10, pdf.get_y() - 10, 0, 10, 'https://www.gob.mx/ActaNacimiento/')
            
            # Añadir el código QR más abajo
            pdf.image(qr_filename, x=10, y=pdf.get_y() + 10, w=100)  # Ajustar posición y tamaño de la imagen
            
            # Guardar el PDF
            pdf.output(f"CodigosQR/curp_{row['id']}.pdf")

            print(f"PDF generado para {row['nombre']} {row['apellido_paterno']} {row['apellido_materno']}")

    finally:
        cursor.close()
        connection.close()

generar_qr_y_pdf()


PDF generado para GAEL QUINTANA ROMERO
